# intermediate_XED_binary
This notebook takes our custom XED binary dataset and trains an intermediate model.

## Imports & Settings

First, update working directory to parent so that we may use our custom functions

In [ ]:
import os
os.chdir('..')
# os.getcwd( )

In [ ]:
import params
from utils import *
from trainer import *

import numpy as np
import pandas as pd
from datasets import load_from_disk

from transformers import RobertaTokenizer, RobertaForSequenceClassification
from torch.utils.data import TensorDataset, DataLoader, RandomSampler

# suppress model warning
from transformers import logging
logging.set_verbosity_error()

# set logging level
import logging
logging.basicConfig(format='%(message)s', level=logging.INFO)

In [ ]:
# set general seeds
set_seeds(1)

# set dataloader generator seed
g = torch.Generator()
g.manual_seed(1)

# set params for this model
params.num_labels = 2
params.output_dir = "model_saves/intermediate_XED_binary_01"
params.dataset_path = "data/inter_XED/itesd_xed_binary_balanced.hf"

# Ensure we're on an ARM environment if necessary.
platform_check()

## Load Data

### Binary XED

In [ ]:
datasets = load_from_disk(params.dataset_path)
datasets

In [ ]:
# we will need to view and prep the datasets
# this is more easily done as dataframes
train_df = datasets['train'].to_pandas()
validate_df = datasets['validation'].to_pandas() 

In [ ]:
# view training dataset
print("train_df Info:")
print(train_df.info())
print("\ntrain_df Value Counts")
print(train_df['label'].value_counts())

In [ ]:
# view validation dataset
print("validate_df Info:")
print(validate_df.info())
print("\n validate_df Value Counts")
print(validate_df['label'].value_counts())

## Preprocess

In [ ]:
train_token_ids, train_attention_masks = encode_text(train_df.text.values)
validate_token_ids, validate_attention_masks = encode_text(validate_df.text.values)

In [ ]:
train_features = []
for i in range(len(train_token_ids)):
    train_features.append({'label': train_df.label.values[i], 
                           'input_ids': train_token_ids[i], 
                           'attention_mask':train_attention_masks[i]})

validate_features = []
for i in range(len(validate_token_ids)):
    validate_features.append({'label': validate_df.label.values[i], 
                              'input_ids': validate_token_ids[i],
                              'attention_mask':validate_attention_masks[i]})

## Data Split
We split the dataset into train (80%) and validation (20%) sets, and wrap them around a torch.utils.data.DataLoader object.

In [ ]:
# Prepare DataLoader
train_dataloader = DataLoader(
            train_features,
            sampler = RandomSampler(train_features),
            batch_size = params.batch_size,
            worker_init_fn=seed_worker,
            generator=g,
            collate_fn=collate
        )

validation_dataloader = DataLoader(
            validate_features,
            sampler = RandomSampler(validate_features),
            batch_size = params.batch_size,
            worker_init_fn=seed_worker,
            generator=g,
            collate_fn=collate
        )

In [ ]:
# view an example from the dataloader
next(iter(train_dataloader))

## Train

Download transformers.RobertaForSequenceClassificatio, which is a RoBERTa model with a linear layer for sentence classification (or regression) on top of the pooled output:

In [ ]:
# Load the RobertaForSequenceClassification model
model = RobertaForSequenceClassification.from_pretrained('roberta-base',
                                                         num_labels = params.num_labels,
                                                         output_attentions = False,
                                                         output_hidden_states = False,
                                                         )

from torchinfo import summary
summary(model, input_size=(1, 512), dtypes=['torch.IntTensor'])

Set model to device, initialize trainer

In [ ]:
model.to(params.device)
print(f"Device: {params.device}")

optimizer = torch.optim.Adam(params=model.parameters(), 
                             lr=params.learning_rate,
                             weight_decay=params.weight_decay) #roberta

trainer = Trainer(model=model,
                  device=params.device,
                  tokenizer=params.tokenizer,
                  train_dataloader=train_dataloader,
                  validation_dataloader=validation_dataloader,
                  epochs=params.epochs,
                  optimizer=optimizer,
                  val_loss_fn=params.val_loss_fn,
                  num_labels=params.num_labels,
                  output_dir=params.output_dir,
                  save_freq=params.save_freq,
                  checkpoint_freq=params.checkpoint_freq)

output_parameters()

Fit the model to our training data.

In [ ]:
trainer.fit()